In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [1]:
from transformers import pipeline

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [12]:
model = AutoModelForCausalLM.from_pretrained(
   'EleutherAI/pythia-2.8b',
    cache_dir = '../hf-models',
).eval()

In [13]:
from peft import PeftModel, PeftConfig

In [14]:
model = PeftModel.from_pretrained(model, "../hf-models/usvsnsp-sft-model")

In [15]:
model = model.cuda()

In [16]:
tokenizer = AutoTokenizer.from_pretrained(
    '../hf-models/usvsnsp-sft-model', 
    cache_dir = '../hf-models',
)

In [35]:
import torch
def generate(text):
    tokens = tokenizer([text], return_tensors = 'pt').input_ids.cuda()
    text = model.generate(
        input_ids = tokens, 
        do_sample = True,
#         num_beams = 5,
        temperature = 0.3,
        top_k = 30,
        top_p = 0.8,
        max_length = 512,
        repetition_penalty = 1.1,
        use_cache = False
    )
    print(tokenizer.decode(text[0]))

In [44]:
generate(
    ("Query: I heard that the use of gram herbicide can cause negative effects on the environment. Can you explain more about this?\n"
    "Response: Gram herbicide is not a specific product name, but a term used to describe herbicides that are applied to crops that have been genetically modified to tolerate them, such as glyphosate, dicamba or 2,4-D. These herbicides can have negative effects on the environment by killing non-target plants and animals, contaminating water sources, reducing biodiversity and increasing the risk of herbicide resistance   . Some of these effects can be mitigated by following proper application guidelines, using alternative weed control methods and choosing environmentally friendly herbicide formulations .\n"
    "Query: {}\n"
    "Response:").format("I am a farmer from Telangana. Summer rains have spoiled my crops. Tell me what should I do now?")
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Query: I heard that the use of gram herbicide can cause negative effects on the environment. Can you explain more about this?
Response: Gram herbicide is not a specific product name, but a term used to describe herbicides that are applied to crops that have been genetically modified to tolerate them, such as glyphosate, dicamba or 2,4-D. These herbicides can have negative effects on the environment by killing non-target plants and animals, contaminating water sources, reducing biodiversity and increasing the risk of herbicide resistance  . Some of these effects can be mitigated by following proper application guidelines, using alternative weed control methods and choosing environmentally friendly herbicide formulations.
Query: I am a farmer from Telangana. Summer rains have spoiled my crops. Tell me what should I do now?
Response: If your crop has been affected by summer rain, it may be necessary to replant the crop in order to recover. The best time for planting is after the last rain

In [6]:
pipe = pipeline('text-generation', model = model, tokenizer = tokenizer, device = 3)

In [9]:
pipe(
    "Hey I am a farm", 
    max_length=128,
)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "Hey I am a farmer but I am not a farmer. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer"}]

In [10]:
with open("../kcc_data/jsonlines/data.jsonl", 'r') as reader:
        data = [i[1:-2] for i in reader]

In [11]:
tokens = tokenizer(
    [data[0]], 
    return_tensors = 'pt',
    truncation = True,
    max_length = 2048
)

In [12]:
input_ids = tokens['input_ids'][:, 0:-1].cuda()
labels = tokens['input_ids'][:, 1:].cuda()

In [ ]:
model = 

In [13]:
input_ids

tensor([[   29,    93,  7267,  ...,   964,   870, 10927]])

In [14]:
labels

tensor([[   93,  7267, 49651,  ...,   870, 10927,  4357]])